In [104]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.optimize as opt
import copy
np.set_printoptions(threshold=np.inf, suppress=True, formatter={'float': '{: 0.8f}'.format}, linewidth=75)

In [105]:
test_matrix = np.array([[1, 2], [3, 4]])
big_test_matrix = la.block_diag(test_matrix, test_matrix)
readable_big_test_matrix = pd.DataFrame(big_test_matrix)
print(type(big_test_matrix))

<class 'numpy.ndarray'>


In [106]:
def kron(A, B):
    return la.kron(A, B)

In [107]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(B, np.eye(A.shape[0]))
    
    return L+R

In [108]:
print(kronsum(big_test_matrix[:2, :2], big_test_matrix[2:4, 2:4]))

[[ 2.00000000  0.00000000  4.00000000  0.00000000]
 [ 0.00000000  2.00000000  0.00000000  4.00000000]
 [ 6.00000000  0.00000000  8.00000000  0.00000000]
 [ 0.00000000  6.00000000  0.00000000  8.00000000]]


In [109]:
n = 1
p_num = 100
eps_G = 10 ** (-8)
eps_Phi = 10 ** (-6)
eps_p_i = 10 ** (-6)

In [110]:
# Входной BMAP
matrD_0 = np.array([[-4.69791416]])
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

for k in range(1, n+1):
    matrD_k.append(-matrD_0)

for matr in matrD_k:
    print(matr)

[[-12.28571429  0.00142857]
 [ 0.00285714 -0.39428571]]
[[ 4.97658080  0.05796253]
 [ 0.01170960  0.14871194]]
[[ 3.98126464  0.04637002]
 [ 0.00936768  0.11896956]]
[[ 3.18501171  0.03709602]
 [ 0.00749415  0.09517564]]


In [111]:
# Характеристики входного BMAP

matrD_1_ = np.zeros(matrD_k[0].shape)
for matr in matrD_k:
    matrD_1_ += matr
print(matrD_1_)
matr_a = copy.deepcopy(matrD_1_)
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

theta = la.solve(matr_a, matr_b).reshape(-1)

matrdD_1_ = copy.deepcopy(matrD_k[1])

for i in range(2, n+1):
    matrdD_1_ += matrD_k[i] * i
vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
lamD = theta.dot(matrdD_1_).dot(vect_e)
print(lamD)

[[-0.14285714  0.14285714]
 [ 0.03142857 -0.03142857]]


[ 4.69791416]


In [112]:
# Поток поломок MAP
matrH0 = np.array([[-6.00065225]])
matrH1 = np.array([[6.00065225]])
V_ = matrH1.shape[0]
V = V_ - 1
matrH = matrH0 + matrH1
matr_a = copy.deepcopy(matrH)
for i in range(matr_a.shape[0]):
    print(matr_a)
    matr_a[i][0] = 1
    
matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

print(matr_a)

gamma = la.solve(matr_a, matr_b).reshape(-1)

vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
h = gamma.dot(matrH1).dot(vect_e)
print(h)

[[-0.05392500  0.05392500]
 [ 0.14662500 -0.14662500]]
[[ 1.00000000  0.05392500]
 [ 0.14662500 -0.14662500]]
[[ 1.00000000  1.00000000]
 [ 0.05392500 -0.14662500]]
[ 6.00065225]


In [113]:
# Поток обслуживания PH1
# TODO make e vectors dynamically allocated!
beta1 = np.matrix([1])
matrS1 = np.matrix([[-10]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.matrix([[1]])
matrS1_0 = - matrS1 * M1_e
vect_e = np.array([[1.] for i in range(0, matrS1.shape[0])])
mu_1 = -la.inv(beta1.dot(la.inv(matrS1)).dot(vect_e))
print(mu_1)

[[ 10.00000000]]


In [114]:
# Поток обслуживания PH2
beta2 = np.matrix([1])
matrS2 = np.matrix([[-1]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.matrix([[1]])
matrS2_0 = - matrS2 * M2_e

vect_e = np.array([[1.] for i in range(0, matrS2.shape[0])])
mu_2 = -la.inv(beta2.dot(la.inv(matrS2)).dot(vect_e))
print(mu_2)

[[ 1.00000000]]


In [115]:
matrS_w = kron(matrS1_0 * beta1, M2_e * beta2) + kron(M1_e * beta1, matrS2_0 * beta2)

In [116]:
# Поток переключения с прибора-1 на прибор-2
alpha1 = np.matrix([1])
matrA1 = np.matrix([[-29.99985287]])
L1 = matrA1.shape[0]
L1_ = L1 + 1
L1_e = np.matrix([[1]])
matrA1_0 = - matrA1.dot(L1_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_1 = -la.inv(alpha1.dot(la.inv(matrA1)).dot(vect_e))
print(kappa_1)

[[ 29.99985287]]


In [117]:
# Поток переключения с прибора-2 на прибор-1
alpha2 = np.matrix([1])
matrA2 = np.matrix([[-89.99955862]])
L2 = matrA2.shape[0]
L2_ = L2 + 1
L2_e = np.matrix([[1]])
matrA2_0 = - matrA2.dot(L2_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_2 = -la.inv(alpha2.dot(la.inv(matrA2)).dot(vect_e))
print(kappa_2)

[[ 89.99955862]]


In [118]:
# Поток ремонта PH
tau = np.matrix([1])
matrT = np.matrix([[-100.0098]])
T_e = np.matrix([[1]])
matrT0 = - matrT.dot(T_e)

R = matrT.shape[0]
R_ = R + 1
vect_e = np.array([[1.] for i in range(0, matrT.shape[0])])
phi = -(tau.dot(la.inv(matrT).dot(vect_e)))
print(phi)

[[ 100.00980000]]


In [119]:
a = W_ * V_

In [120]:
# Q~0
block00 = kronsum(matrD_k[0], matrH0)
block03 = kron(kron(kron(np.eye(W_), matrH1), tau), alpha1)
block10 = kron(np.eye(a), matrA2_0)
block11 = kronsum(kronsum(matrD_k[0], matrH0), matrA2)
block12 = kron(kron(kron(np.eye(W_), matrH1), tau), L2_e)
block21 = kron(kron(np.eye(a), matrT0), alpha2)
block22 = kronsum(kronsum(matrD_k[0], matrH), matrT)
block30 = kron(kron(np.eye(a), matrT0), L1_e)
block32 = kron(kron(np.eye(a), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

print(block00.shape[1] + block11.shape[1] + block22.shape[1] + block33.shape[1])

matrQw_0 = np.bmat([[block00, block01, block02, block03],
                    [block10, block11, block12, block13], 
                    [block20, block21, block22, block23],
                    [block30, block31, block32, block33]])
print(matrQw_0)

36
[[-20.39643929 -0.00000000  0.00142857  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.39478320  7.50088080  0.00805680  0.15307920
   0.00264233  0.05020417  0.00005393  0.00102458  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [-0.00000000 -20.39643929  0.00000000  0.00142857  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00718463  0.13650788  0.00014663  0.00278588
   0.00571463  0.10857788  0.00011663  0.00221588  0.00000000  0.00000000
   0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000]
 [ 0.00285714  0.00000000 -0.65753571 -0.00000000  0.00000000  0.00000000
   0.00000000  0.00000000  0.00

In [121]:
# Q~k
matrQw_k = [matrQw_0]
for i in range(1, n+1):
    block0 = kron(kron(matrD_k[i], np.eye(V_)), beta1)
    block1 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(L2))
    block2 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(R))
    block3 = kron(kron(matrD_k[i], np.eye(V_)), np.eye(R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQw_k.append(matr_temp)
pd.DataFrame(matrQw_k[1])
sum = 0
for i in range(matrQw_k[0].shape[0]):
    for matr in matrQw_k:
        sum += np.sum(matr[i])
print('matrQw_k[' + str(i) + '] = ', sum)

matrQw_k[35] =  1.88404847279e-13


In [122]:
# Q^0
block0 = kron(np.eye(a), matrS1_0)
block1 = kron(kron(np.eye(a), matrS2_0), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQv_0 = la.block_diag(block0, block1, block2, block3)
pd.DataFrame(matrQv_0)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
# Q_0
block0 = kron(np.eye(a), matrS1_0.dot(beta1))
block1 = kron(kron(np.eye(a), matrS2_0.dot(beta2)), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0.dot(beta2)), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQ_0 = la.block_diag(block0, block1, block2, block3)
pd.DataFrame(matrQ_0)

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
# Q_1
block00 = kronsum(kronsum(matrD_k[0], matrH0), matrS1)
block03 = kron(kron(kron(kron(np.eye(W_), matrH1), M1_e), tau), alpha1)
block10 = kron(kron(kron(np.eye(a), M2_e), beta1), matrA2_0)
block11 = kronsum(kronsum(kronsum(matrD_k[0], matrH0), matrS2), matrA2)
block12 = kron(kron(kron(kron(np.eye(W_), matrH1), np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(a), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrS2), matrT)
block30 = kron(kron(kron(np.eye(a), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(a), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

matrQ_1 = np.bmat([[block00, block01, block02, block03], 
                   [block10, block11, block12, block13], 
                   [block20, block21, block22, block23], 
                   [block30, block31, block32, block33]])
pd.DataFrame(matrQ_1)

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-40.396439,-0.000000,-0.000000,-0.000000,20.001429,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000054,0.001025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000000,-40.396439,-0.000000,-0.000000,0.000000,20.001429,0.000000,0.000000,0.000000,0.000000,...,0.000054,0.001025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.000000,-0.000000,-40.396439,-0.000000,0.000000,0.000000,20.001429,0.000000,0.000000,0.000000,...,0.000117,0.002216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-0.000000,-0.000000,-0.000000,-40.396439,0.000000,0.000000,0.000000,20.001429,0.000000,0.000000,...,0.000117,0.002216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.002857,0.000000,0.000000,0.000000,-20.657536,-0.000000,-0.000000,-0.000000,0.000000,0.000000,...,0.000000,0.000000,0.394783,7.500881,0.008057,0.153079,0.002642,0.050204,0.000054,0.001025
5,0.000000,0.002857,0.000000,0.000000,-0.000000,-20.657536,-0.000000,-0.000000,0.000000,0.000000,...,0.000000,0.000000,0.394783,7.500881,0.008057,0.153079,0.002642,0.050204,0.000054,0.001025
6,0.000000,0.000000,0.002857,0.000000,-0.000000,-0.000000,-20.657536,-0.000000,0.000000,0.000000,...,0.000000,0.000000,0.007185,0.136508,0.000147,0.002786,0.005715,0.108578,0.000117,0.002216
7,0.000000,0.000000,0.000000,0.002857,-0.000000,-0.000000,-0.000000,-20.657536,0.000000,0.000000,...,0.000000,0.000000,0.007185,0.136508,0.000147,0.002786,0.005715,0.108578,0.000117,0.002216
8,5.582250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-27.978689,-0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,440.998200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-27.978689,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [125]:
matrQ_k = [matrQ_0, matrQ_1]
for k in range(2, n+1):
    block0 = kron(matrD_k[k-1], np.eye(V_ * M1))
    block1 = kron(matrD_k[k-1], np.eye(V_*M2*L2))
    block2 = kron(matrD_k[k-1], np.eye(V_*M2*R))
    block3 = kron(matrD_k[k-1], np.eye(V_*R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQ_k.append(matr_temp)

print(matrQ_k[2][0])
sum = 0
for i in range(matrQ_k[0].shape[0]):
    sum += np.sum(matrQv_0[i])
    for j in range(1, np.array(matrQ_k).shape[0]):
        sum += np.sum(matrQ_k[j][i])
print('matrQ_k[' + str(i) + '] = ', sum)

[ 4.97658080  0.00000000  0.00000000  0.00000000  0.05796253  0.00000000
  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000  0.00000000
  0.00000000  0.00000000]
matrQ_k[55] =  -93.0937704918


In [126]:
# Проверка генератора Q

In [126]:
# Поиск матрицы G

matrQ_1_neg_inv = np.linalg.inv(-matrQ_k[1])

def calc_G(matrG_prev):
    temp_sum = matrQ_k[0].copy()
    for k in range(1, n+1):
        temp_sum += matrQ_k[k].dot(np.linalg.matrix_power(matrG_prev, k))
    matrG_new = matrQ_1_neg_inv.dot(temp_sum)
    return matrG_new
        
matrG_old = np.eye(matrQ_k[1].shape[0])
matrG = calc_G(matrG_old)
i = 1
while la.norm(matrG - matrG_old, ord=np.inf) >= eps_G:
    matrG_old = matrG
    matrG = calc_G(matrG_old)
    i += 1
    print(i,  'norm = ', la.norm(matrG - matrG_old, ord=np.inf))

pd.DataFrame(matrG)

2 norm =  11.569150941096485
3 norm =  10.967847016592843
4 norm =  21.135151532937403
5 norm =  88.58046699007828
6 norm =  5298.585728392576
7 norm =  1521120628.5397334
8 norm =  4.126256043479468e+25
9 norm =  7.347609781062842e+74
10 norm =  4.6852703231036746e+222


ValueError: array must not contain infs or NaNs

In [ ]:
for i in range(matrQw_k[0].shape[0]):
    sum = 0
    for matr in matrQw_k:
        sum += np.sum(matr[i])
    print('Sum matrQw_k[' + str(i) + '] = ', sum)

for i in range(matrQ_k[0].shape[0]):
    sum = np.sum(matrQv_0[i])
    for matr in matrQ_k:
        sum += np.sum(matr[i])
    print('Sum matrQ_k[' + str(i) + '] = ', sum)

In [ ]:
temp_sum = matrQ_k[1].copy()
for k in range(2, n+1):
    temp_sum += matrQ_k[k].dot(np.linalg.matrix_power(matrG, k-1))
matrG_0 = la.inv(temp_sum)
matrG_0 = (-matrG_0).dot(matrQv_0)
pd.DataFrame(matrG_0)

In [ ]:
matrQ_il = []
for i in range(0, p_num):
    matrQ_il.append([])
    if i == 0:
        for l in range(0, n+1):
            # здесь до n, т.к. нет больше матриц Q_k
            temp_matr = matrQw_k[l].copy()
            for k in range(l+1, n+1):
                mult_matr = matrQw_k[k].copy()
                for kk in range(k-1, l-1, -1):
                    if kk == 0:
                        mult_matr = mult_matr.dot(matrG_0)
                    else:
                        mult_matr = mult_matr.dot(matrG)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
        for l in range(n+1, p_num):
            matrQ_il[i].append(np.zeros(matrQ_il[i][l % n+1].shape)) 
    else:
        for l in range(0, p_num):
            if l >= i and l - i <= n:
                temp_matr = matrQ_k[l-i].copy()
                for k in range(l+1, n+1):
                    if (k - i <= 3):
                        mult_matr = matrQ_k[k-i].copy()
                        for kk in range(k-1, l-1, -1):
                            if kk == 0:
                                mult_matr = mult_matr.dot(matrG_0)
                            else:
                                mult_matr = mult_matr.dot(matrG)
                        temp_matr += mult_matr
                matrQ_il[i].append(temp_matr)
            else:
                matrQ_il[i].append(np.zeros(matrQ_k[0].shape))
    
print(len(matrQ_il[3]))
pd.DataFrame(la.inv(-matrQ_il[1][1]))

In [ ]:
matrPhi_0 = np.eye(matrQ_il[0][0].shape[0])
matrPhi_l = [matrPhi_0]
for l in range(1, p_num):
    temp_matr = matrPhi_l[0].dot(matrQ_il[0][l]).dot(la.inv(-matrQ_il[l][l]))
    for i in range(1, l):
        # print(matrPhi_l[i].dot(matrQ_il[i][l]).dot(la.inv(-matrQ_il[l][l])).shape)
        temp_matr += (matrPhi_l[i].dot(matrQ_il[i][l])).dot(la.inv(-matrQ_il[l][l]))
    matrPhi_l.append(temp_matr)
print(la.norm(matrPhi_l[25], np.inf))

In [ ]:
matr_a = np.transpose(- matrQ_il[0][0])
matr_b = np.zeros((matr_a.shape[0], 1))
p0 = la.lstsq(matr_a, matr_b)[3]
p0 = np.transpose(p0)

print(type(a), type(R_))

vect_e1 = np.array([[1.] for i in range(0, p0.shape[0])])
print(vect_e1)
temp_vect = vect_e1.copy()
for l in range(1, p_num):
    vect_e2 = np.array([[1.] for i in range(0, matrPhi_l[l].shape[1])])
    temp_vect += matrPhi_l[l].dot(vect_e2)
res = p0.dot(temp_vect)[0]
print('temp_vect = ', temp_vect)
p0 /= res
print(p0.sum())

In [ ]:
vect_p_l = [p0]
for l in range(1, n+1):
    vect_p_l.append(vect_p_l[0].dot(matrPhi_l[l]))
    print(vect_p_l[l])

In [ ]:
# Условие эргодичности
block00 = kronsum(matrH0, matrS1) + kron(np.eye(V_), matrS1_0.dot(beta1))
block03 = kron(kron(kron(matrH1, M1_e), tau), alpha1)
block10 = kron(kron(np.eye(V_), M2_e.dot(beta1)), matrA2_0)
block11 = kronsum(kronsum(matrH0, matrS2), matrA2) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(L2))
block12 = kron(kron(kron(matrH1, np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(V_), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(matrH, matrS2), matrT) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(R))
block30 = kron(kron(kron(np.eye(V_), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(V_), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(matrH, matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block00.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))
matrGamma = np.bmat([[block00, block01, block02, block03],
                     [block10, block11, block12, block13],
                     [block20, block21, block22, block23],
                     [block30, block31, block32, block33]])
pd.DataFrame(matrGamma)

In [ ]:
matr_a = np.transpose(matrGamma)
matr_b = np.zeros((matr_a.shape[0], 1))
x = la.lstsq(matr_a, matr_b)[3]
vect_e = np.array([[1.] for i in range(0, x.shape[0])])
res = x.dot(vect_e)[0]
x /= res
print('x = ', x)
x1 = x[0:V_*M1]
x2 = x[V_*M1:V_*M1 + V_*M2*L2]
x3 = x[V_*M1 + V_*M2*L2: V_*M1 + V_*M2*L2 + V_*M2*R]

print('x1 = ', x1)
print('x2 = ', x2)
print('x3 = ', x3)

In [ ]:
e_V_ = np.array([[1.] for i in range(0, V_)])
e_R = np.array([[1.] for i in range(0, R)])
pi1 = x1.dot(kron(e_V_, np.eye(M1)))
pi2 = x2.dot(kron(kron(e_V_, np.eye(M2)), L2_e))
pi3 = x3.dot(kron(kron(e_V_, np.eye(M2)), e_R))
print('pi1 = ', pi1)
print('pi2 = ', pi2)
print('pi3 = ', pi3)

In [ ]:
rho = pi1.dot(matrS1_0) + (pi2 + pi3).dot(matrS2_0)
rho = rho.tolist()[0][0]
print('rho = ', rho)

In [ ]:
print(la.norm(np.eye(matrQ_il[0][0].shape[0]).dot(matrQ_il[0][1]).dot(la.inv(-matrQ_il[1][1])), np.inf))

In [ ]:
result = np.sum(matrQv_0[0])
i = 0
for matrQk in matrQ_k:
    if i > 0:
        result += np.sum(matrQk[0])
    i += 1
print(result)